# Look at cell distances between different populations

In [ ]:
import os
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist

In [ ]:
cell_tab_path = "../data/tables/cell_table_size_normalized.csv"
functional_tab_path = "../data/tables/nimbus_binarized.csv"

output_dir = "../data/spatial_analysis/cell_cell_distances_functional"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

cell_table = pd.read_csv(cell_tab_path)
cell_table = cell_table[['fov','label','centroid-0','centroid-1']]
func_table = pd.read_csv(functional_tab_path)
cell_table = cell_table.merge(func_table, on=['fov','label'], how='inner')

all_fovs = np.unique(cell_table['fov'])

In [ ]:
one_pheno = "CD8T"
functional_marker = "Caspase1"

cell_table['new_pheno'] = cell_table['cell_meta_cluster'] + "_" + cell_table[functional_marker].astype(str)

In [ ]:
all_pheno1s = [one_pheno+"_0",one_pheno+"_1"]
all_pheno2s = [one_pheno+"_0",one_pheno+"_1"]
pheno_list = [(pheno1, pheno2) for pheno1 in all_pheno1s for pheno2 in all_pheno2s]

for pheno1,pheno2 in pheno_list:
    data_list = []
    for fov in all_fovs:
        # Calculate distance between all cells in the FOV
        fov_cell_table = cell_table.loc[cell_table['fov'] == fov].reset_index(drop=True)
        all_centroids = list(zip(fov_cell_table['centroid-0'],fov_cell_table['centroid-1']))
        dist_mat = cdist(all_centroids, all_centroids, 'euclidean')

        # Count number of close contacts between cell types
        pheno1_idx = fov_cell_table[fov_cell_table['new_pheno'] == pheno1].index.to_list()
        pheno2_idx = fov_cell_table[fov_cell_table['new_pheno'] == pheno2].index.to_list()

        # Check if the cell type exists in the data
        if len(pheno1_idx)==0 or len(pheno2_idx)==0:
            data_list.append([pheno1, pheno2, fov, ""])
            continue

        # Subset distance matrix
        keep_dist = dist_mat[pheno1_idx,:]
        keep_dist = keep_dist[:,pheno2_idx]

        # Remove duplicates if pheno1==pheno2
        if pheno1 == pheno2:
            row_indices, col_indices = np.triu_indices(len(pheno1_idx), k=1)
            keep_dist = keep_dist[row_indices, col_indices]

        data_list.append([pheno1, pheno2, fov, np.mean(keep_dist)])

    save_df = pd.DataFrame(data_list, columns=['pheno1', 'pheno2', 'fov', 'mean'])
    save_df.to_csv(os.path.join(output_dir, pheno1+"_"+pheno2+"_"+functional_marker+".csv"), index=False)